# Monthly Retrospective script
The following script will produce a list of statistics that might be relevant for the Monthly retrospective. **This new version uses the Geonet API and can therefore be run on the first!**

All you need to do is change the dates to the correct month in the box below and select "run all cells".

This is a work in progress and would appreciate any feedback as to what other information, graphs etc would be useful. 

In [ ]:
import requests
import pandas as pd

## Times
Change the date here. Because it is in UTC time, start it on the last day of the previous month and end it on the last of the current month. 
During daylight savings time (Summer), midnight NZT is at at 11:00:00, otherwise set it to 12:00:00

In [ ]:
starttime = "2019-12-31T11:00:00"
enddate = "2020-01-31T11:00:00"

The following section is the code that will automatically run with the above input. The output is found at the bottom.

In [ ]:
jase = requests.get("http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=geonet:quake_search_v1&outputFormat=json&cql_filter=origintime>=" 
                    + starttime + "+AND+origintime<=" +enddate).json()

In [ ]:

lat, lon, mag, quakeid = [[] for x in range(4)]
for a in range(len(jase['features'])):
    lon.append(jase['features'][a]['geometry']['coordinates'][0])
    lat.append(jase['features'][a]['geometry']['coordinates'][1])
    mag.append(jase['features'][a]['properties']['magnitude'])
    quakeid.append(jase['features'][a]['properties']['publicid'])
df = pd.DataFrame({"latitude":lat,"longitude":lon,"magnitude":mag,"quake_id":quakeid})
magsum = pd.DataFrame({})
magsum['magnitudes'] = ["0-1", "1-2","2-3","3-4","4-5","5-6","6-7","7-8","8+"]
magsum['count'] = (df['magnitude'].value_counts(bins = [0,1,2,3,4,5,6,7,8,9]))
magsum = magsum.sort_index()
magsum = magsum.set_index('magnitudes')
print(magsum)
kermadec = df[(df['latitude'] <=-25) & (df['latitude']>=-33) & (df['longitude'] >= 177) & (df['longitude'] <=180) | 
             (df['latitude'] <=-25) & (df['latitude']>=-33) & (df['longitude'] >= -180) & (df['longitude'] <=-169)]
local = df[(df['latitude'] <=-33) & (df['latitude'] >= -50) & (df['longitude'] >= 163) & (df['longitude'] <=180) | 
          (df['latitude'] <=-33) & (df['latitude'] >= -50) & (df['longitude'] >= -180) & (df['longitude'] <=-175)|
          (df['latitude'] <=-32) & (df['latitude'] >= -33) & (df['longitude'] >= 163) & (df['longitude'] <=177)]
print("\nTotal quakes: ", len(df), "\nLocal", len(local), "\nKermadec: ", len(kermadec), "\nTeleseismics: ", len(df) - len(kermadec) - len(local),
     "\nMaxmimum magnitude", df['magnitude'].max(), " ID: ", df.iloc[df['magnitude'].idxmax()]['quake_id'],
      "\nMinimum magnitude", df['magnitude'].min(), "ID: ", df.iloc[df['magnitude'].idxmin()]['quake_id'])

